# Preparação do Dataset para Tableau

Este notebook realiza transformações no dataset "Student Academic Performance" para otimizá-lo para uso no Tableau. As transformações incluem:

- Padronização de textos categóricos para português.
- Criação de novas colunas (feature engineering): Categoria_Desempenho e Faixa_Frequencia.
- Normalização de variáveis numéricas (z-score para Horas_Estudadas).

Ao final, o dataset transformado é salvo como CSV para importação no Tableau.

In [ ]:
# Import Required Libraries
# Importa pandas para manipulação de dados (parâmetro: pd como alias)
import pandas as pd
# Importa numpy para operações numéricas (parâmetro: np como alias)
import numpy as np
# Importa StandardScaler do sklearn para normalização z-score (parâmetro: usado para escalar dados)
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load Data
# Carrega o dataset CSV original (parâmetro: caminho do arquivo)
df = pd.read_csv('../student_academic_performance/StudentPerformanceFactors.csv')
# Imprime mensagem de confirmação com shape do dataframe (linhas, colunas)
print("Dataset carregado. Shape:", df.shape)
# Imprime as primeiras 5 linhas do dataframe para visualização
print(df.head())

Dataset carregado. Shape: (6607, 20)
   Hours_Studied  Attendance Parental_Involvement Access_to_Resources  \
0             23          84                  Low                High   
1             19          64                  Low              Medium   
2             24          98               Medium              Medium   
3             29          89                  Low              Medium   
4             19          92               Medium              Medium   

  Extracurricular_Activities  Sleep_Hours  Previous_Scores Motivation_Level  \
0                         No            7               73              Low   
1                         No            8               59              Low   
2                        Yes            7               91           Medium   
3                        Yes            8               98           Medium   
4                        Yes            6               65           Medium   

  Internet_Access  Tutoring_Sessions Family_Incom

In [ ]:
# Data Cleaning
# Imprime mensagem para verificar valores nulos
print("Verificando nulos:")
# Conta e imprime o número de valores nulos por coluna
print(df.isnull().sum())
# Loop para preencher nulos em colunas categóricas específicas
for col in ['Teacher_Quality', 'Parental_Education_Level', 'Distance_from_Home']:
    # Verifica se há nulos na coluna
    if df[col].isnull().sum() > 0:
        # Calcula a moda (valor mais frequente) da coluna
        mode_val = df[col].mode()[0]
        # Preenche nulos com a moda (parâmetro: inplace=True modifica o dataframe original)
        df[col].fillna(mode_val, inplace=True)
# Imprime confirmação de preenchimento
print("Nulos preenchidos.")

Verificando nulos:
Hours_Studied                  0
Attendance                     0
Parental_Involvement           0
Access_to_Resources            0
Extracurricular_Activities     0
Sleep_Hours                    0
Previous_Scores                0
Motivation_Level               0
Internet_Access                0
Tutoring_Sessions              0
Family_Income                  0
Teacher_Quality               78
School_Type                    0
Peer_Influence                 0
Physical_Activity              0
Learning_Disabilities          0
Parental_Education_Level      90
Distance_from_Home            67
Gender                         0
Exam_Score                     0
dtype: int64
Nulos preenchidos.


/var/folders/2g/3jr0tfzj3979tfb0cvjqnk700000gn/T/ipykernel_3492/4192249349.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_val, inplace=True)


In [ ]:
# Data Transformations

# Dicionário de mapeamentos para traduzir valores categóricos para português
mappings = {
    'Parental_Involvement': {'Low': 'Baixo', 'Medium': 'Médio', 'High': 'Alto'},
    'Access_to_Resources': {'Low': 'Baixo', 'Medium': 'Médio', 'High': 'Alto'},
    'Motivation_Level': {'Low': 'Baixo', 'Medium': 'Médio', 'High': 'Alto'},
    'Internet_Access': {'Yes': 'Sim', 'No': 'Não'},
    'Extracurricular_Activities': {'Yes': 'Sim', 'No': 'Não'},
    'Learning_Disabilities': {'Yes': 'Sim', 'No': 'Não'},
    'Family_Income': {'Low': 'Baixo', 'Medium': 'Médio', 'High': 'Alto'},
    'Teacher_Quality': {'Low': 'Baixo', 'Medium': 'Médio', 'High': 'Alto'},
    'School_Type': {'Public': 'Público', 'Private': 'Privado'},
    'Peer_Influence': {'Positive': 'Positivo', 'Negative': 'Negativo', 'Neutral': 'Neutro'},
    'Parental_Education_Level': {'High School': 'Ensino Médio', 'College': 'Faculdade', 'Postgraduate': 'Pós-Graduação'},
    'Distance_from_Home': {'Near': 'Perto', 'Moderate': 'Moderado', 'Far': 'Longe'},
    'Gender': {'Male': 'Masculino', 'Female': 'Feminino'}
}

# Loop para aplicar os mapeamentos a cada coluna
for col, mapping in mappings.items():
    # Verifica se a coluna existe no dataframe
    if col in df.columns:
        # Aplica o mapeamento (tradução) à coluna
        df[col] = df[col].map(mapping)

# Feature Engineering: Cria coluna Categoria_Desempenho baseada em Exam_Score
# pd.cut divide em bins (parâmetros: bins para intervalos, labels para nomes)
df['Categoria_Desempenho'] = pd.cut(df['Exam_Score'], bins=[0, 65, 80, 100], labels=['Baixo', 'Médio', 'Alto'])

# Feature Engineering: Cria coluna Faixa_Frequencia baseada em Attendance
# pd.cut com bins para faixas de frequência
df['Faixa_Frequencia'] = pd.cut(df['Attendance'], bins=[0, 60, 80, 100], labels=['Baixa', 'Média', 'Alta'])

# Normalização: Aplica z-score à coluna Horas_Estudadas
# StandardScaler() cria o escalador
scaler = StandardScaler()
# fit_transform calcula média/desvio e transforma (parâmetro: array da coluna)
df['Horas_Estudadas_Normalizado'] = scaler.fit_transform(df[['Hours_Studied']])

# Imprime confirmação das transformações aplicadas
print("Transformações aplicadas. Shape final:", df.shape)
# Imprime as primeiras linhas para verificar
print(df.head())

Transformações aplicadas. Shape final: (6607, 23)
   Hours_Studied  Attendance Parental_Involvement Access_to_Resources  \
0             23          84                Baixo                Alto   
1             19          64                Baixo               Médio   
2             24          98                Médio               Médio   
3             29          89                Baixo               Médio   
4             19          92                Médio               Médio   

  Extracurricular_Activities  Sleep_Hours  Previous_Scores Motivation_Level  \
0                        Não            7               73            Baixo   
1                        Não            8               59            Baixo   
2                        Sim            7               91            Médio   
3                        Sim            8               98            Médio   
4                        Sim            6               65            Médio   

  Internet_Access  Tutoring_Sessions

In [ ]:
# Save Transformed Data for Tableau
# Salva o dataframe transformado em CSV (parâmetros: caminho do arquivo, index=False para não salvar índices)
df.to_csv('../student_academic_performance/StudentPerformanceFactors_transformado.csv', index=False)
# Imprime confirmação com o caminho do arquivo salvo
print("Arquivo salvo: ../student_academic_performance/StudentPerformanceFactors_transformado.csv")

Arquivo salvo: ../student_academic_performance/StudentPerformanceFactors_transformado.csv
